A way to get around R memory limits

In [1]:
#from parcels import FieldSet, ParticleSet, JITParticle
#from parcels import AdvectionRK4
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np
#from datetime import timedelta as delta
import netCDF4
#import xarray as xr
from sklearn.neighbors import BallTree
from csv import writer
import os

# Code for nearest neighboor if large datasets that n100 code can't handle

In [2]:
# load in metadata for island codes 1-32
names = pd.read_csv('IslandHabitatPixels.csv', names = ['IslandName','IslandNumber','NumberOfHabitatPixels'])

In [3]:
# download land coordinates from csv file
land = pd.read_csv('HIreefsNew.csv', names=['lat', 'lon', 'cover', 'island'])
for column in land[["lat", "lon"]]:
    rad = np.deg2rad(land[column].values)
    land[f'{column}_rad'] = rad

In [4]:
# use this function to find closest island to each recorded location for each particle
ball = BallTree(land[["lat_rad", "lon_rad"]].values, metric='haversine')

In [5]:
def nearest_neighbor(model_data, presettlement,PLD, settle_dist):
    # load the data
    lon = model_data.variables['lon'][:]
    lat = model_data.variables['lat'][:]
    age= model_data.variables['age'][:]
    site = model_data.variables['releaseSite'][:]
    trajectory = model_data.variables['trajectory'][:]
    
    # this just retrieves a label for each particle
    trajectory = trajectory[:,presettlement:PLD]
    labeled = np.reshape(trajectory, -1)
    trajectory = pd.DataFrame(trajectory)
    particle = trajectory.iloc[:, 0]
    del(trajectory)
    
    # remove the first 15 columns because those are before settling age and remove
    lon = lon[:,presettlement:PLD]
    lat = lat[:,presettlement:PLD]
    age = age[:,presettlement:PLD]
    
    site = pd.DataFrame(site)
    site_label = site.iloc[:, 0]
    del(site)
    
    #replace numbers with names of places
    site_label = site_label.replace(to_replace =1,value ="Guam")
    site_label = site_label.replace(to_replace =2,value ="Saipan")
    site_label = site_label.replace(to_replace =3,value ="Majuro")
    site_label = site_label.replace(to_replace =4,value ="Rongelap")
    site_label = site_label.replace(to_replace =5,value ="Kiritimati")
    site_label = site_label.replace(to_replace =6,value ="Pohnpei")
    site_label = site_label.replace(to_replace =7,value ="Okinawa")
    site_label = site_label.replace(to_replace =10,value ="FrenchFrigateShoal")
    site_label = site_label.replace(to_replace =11,value ="Johnston")
    site_label = site_label.replace(to_replace =12,value ="Wake")
    
    # convert to vectors
    lat = np.reshape(lat, -1)
    lon = np.reshape(lon, -1)
    age = np.reshape(age, -1)
    # convert to days from seconds
    age = age/86400
    
    # convert coordinates to radians
    rad_lon = np.deg2rad(lon)
    rad_lat = np.deg2rad(lat)
    del(lon)
    del(lat)
    
    # combine
    combined = np.vstack((rad_lon, rad_lat,labeled)).T
    del(rad_lat)
    del(rad_lon)
    del(labeled)
    coord = pd.DataFrame(data=combined, columns=['lat','lon','particle'])
    del(combined)
    coord['age'] = age
    del(age)
    
    # remove NAs so the next steps work
    coord = coord.dropna()
    
    # NOW, actually start search for nearest neighbor
    distances, indices = ball.query(coord[["lat", "lon"]].values, k = 1)
    
    # add data to dataframe
    coord['closest_point'] = indices
    coord['closest_distance'] = distances
    
    closest = coord.loc[coord.groupby('particle')['closest_distance'].idxmin()]
    del(coord)
    
    site_data = pd.DataFrame(data=[particle,site_label]).T
    del(site_label)
    del(particle)
    site_data.columns = ['particle','release_site']
    merge = closest.set_index('particle').join(site_data.set_index('particle'))
    
    #convert raidians to km
    merge['closest_dist_km'] = 6371 * merge['closest_distance']
    
    # add column of island code for settled
    points = merge['closest_point'].values
    island = []
    for x in points:
        add = land.loc[x,'island']
        island.append(add)
    
    # convert island code to name
    island_names = []
    for x in island:
        add = names.loc[x,'IslandName']
        island_names.append(add)
    merge['closest_island'] = island_names
    
    #make a column of settled or not if = or less than 5km and then graph the results and get summary statistics
    merge.loc[merge['closest_dist_km'] <= settle_dist, 'Settled'] = 'Settled' 
    merge.loc[merge['closest_dist_km'] > settle_dist, 'Settled'] = 'Not close enough'
    #merge.to_csv('/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site.csv', sep='\t')
    #settled_per_release_site = merge.groupby(['release_site', 'Settled']).size().reset_index(name='count')
    #settled_per_release_site.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site.csv', sep='\t')
    #settled_per_settled_site = merge.groupby(['closest_island', 'Settled']).size().reset_index(name='count')
    #settled_per_settled_site.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site.csv', sep='\t')
    
    # make dataset for each settled island to see where the ones settling are coming from
    settled_per_release_and_settledsite = merge.groupby(['release_site','closest_island', 'Settled']).size().reset_index(name='count')
    settled_per_release_and_settledsite.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv', sep='\t')
    #only_settled_per_release_and_settledsite = settled_per_release_and_settledsite[settled_per_release_and_settledsite['Settled'] == 'Settled'] 
    #only_settled_per_release_and_settledsite = only_settled_per_release_and_settledsite.drop(columns=['Settled'])
    
    #correct_format_to_plot_settlement = only_settled_per_release_and_settledsite.pivot_table(index="release_site", columns='closest_island', values='count')
    #correct_format_to_plot_settlement = correct_format_to_plot_settlement.fillna(0)
    
    #fig1, axs = plt.subplots(figsize=(20, 12))
    #ax = correct_format_to_plot_settlement.plot.bar(rot=0, ax=axs)
    #axs.set_ylabel("count")
    #axs.set_title('settlement to each island by release site')
    #fig1.savefig("/Users/taylorely/Documents/OceanModeling/settlement_n500_site.png")
    
    #fig2, axs = plt.subplots(figsize=(8, 50))
    #axes = correct_format_to_plot_settlement.plot.bar(rot=0, subplots=True, ax=axs)
    #axes[1].legend(loc=2)  
    #fig2.savefig("/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site.png")


In [6]:
n100 = netCDF4.Dataset('LAPS_sensitivity_n100_pld180_5m_Kh10_nday545_daily_07012012.nc')
nearest_neighbor(n100, 15,30,10)
#os.rename('/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site.csv', '/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n100.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n100.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n100.csv')
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n100_PLD30_settledist10.csv')
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_n100.png")
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_subplot_n100.png")

In [ ]:
nearest_neighbor(n100, 15,60,10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n100_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor(n100, 15,90,10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n100_PLD90_settledist10.csv')

In [ ]:
nearest_neighbor(n100, 15,180,10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n100_PLD180_settledist10.csv')
del(n100)

In [ ]:
n100_repeat = netCDF4.Dataset('LAPS_sensitivity_n100_pld180_5m_Kh10_nday545_daily_07012012_repeattest.nc')
nearest_neighbor(n100_repeat, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n100_repeat_PLD30_settledist10.csv')
nearest_neighbor(n100_repeat, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n100_repeat_PLD60_settledist10.csv')
nearest_neighbor(n100_repeat, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n100_repeat_PLD90_settledist10.csv')
nearest_neighbor(n100_repeat, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n100_repeat_PLD180_settledist10.csv')
del(n100_repeat)

In [ ]:
subset_1 = netCDF4.Dataset('n500_1.nc')
nearest_neighbor(subset_1, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site1_PLD30_settledist10.csv')
nearest_neighbor(subset_1, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site1_PLD60_settledist10.csv')
nearest_neighbor(subset_1, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site1_PLD90_settledist10.csv')
nearest_neighbor(subset_1, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site1_PLD180_settledist10.csv')
del(subset_1)

In [ ]:
subset_2 = netCDF4.Dataset('n500_2.nc')
nearest_neighbor(subset_2, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site2_PLD30_settledist10.csv')
nearest_neighbor(subset_2, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site2_PLD60_settledist10.csv')
nearest_neighbor(subset_2, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site2_PLD90_settledist10.csv')
nearest_neighbor(subset_2, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site2_PLD180_settledist10.csv')
del(subset_2)
#os.rename('/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site.csv', '/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site2.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site2.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site2.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site2.csv')
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_n500_site2.png")
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site2.png")

In [ ]:
subset_3 = netCDF4.Dataset('n500_3.nc')
nearest_neighbor(subset_3, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site3_PLD30_settledist10.csv')
nearest_neighbor(subset_3, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site3_PLD60_settledist10.csv')
nearest_neighbor(subset_3, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site3_PLD90_settledist10.csv')
nearest_neighbor(subset_3, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site3_PLD180_settledist10.csv')
del(subset_3)

#os.rename('/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site.csv', '/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site3.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site3.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site3.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site3.csv')
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_n500_site3.png")
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site3.png")


In [ ]:
subset_4 = netCDF4.Dataset('n500_4.nc')
nearest_neighbor(subset_4, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site4_PLD30_settledist10.csv')
nearest_neighbor(subset_4, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site4_PLD60_settledist10.csv')
nearest_neighbor(subset_4, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site4_PLD90_settledist10.csv')
nearest_neighbor(subset_4, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site4_PLD180_settledist10.csv')
del(subset_4)
#os.rename('/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site.csv', '/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site4.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site4.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site4.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site4.csv')
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_n500_site4.png")
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site4.png")

In [ ]:
subset_5 = netCDF4.Dataset('n500_5.nc')
nearest_neighbor(subset_5, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site5_PLD30_settledist10.csv')
nearest_neighbor(subset_5, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site5_PLD60_settledist10.csv')
nearest_neighbor(subset_5, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site5_PLD90_settledist10.csv')
nearest_neighbor(subset_5, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site5_PLD180_settledist10.csv')
del(subset_5)

#os.rename('/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site.csv', '/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site5.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site5.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site5.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site5.csv')
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_n500_site5.png")
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site5.png")

In [ ]:
subset_6 = netCDF4.Dataset('n500_6.nc')
nearest_neighbor(subset_6, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site6_PLD30_settledist10.csv')
nearest_neighbor(subset_6, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site6_PLD60_settledist10.csv')
nearest_neighbor(subset_6, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site6_PLD90_settledist10.csv')
nearest_neighbor(subset_6, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site6_PLD180_settledist10.csv')
del(subset_6)

#os.rename('/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site.csv', '/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site6.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site6.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site6.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site6.csv')
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_n500_site6.png")
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site6.png")


In [ ]:
subset_10 = netCDF4.Dataset('n500_10.nc')
nearest_neighbor(subset_10, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site10_PLD30_settledist10.csv')
nearest_neighbor(subset_10, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site10_PLD60_settledist10.csv')
nearest_neighbor(subset_10, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site10_PLD90_settledist10.csv')
nearest_neighbor(subset_10, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site10_PLD180_settledist10.csv')
del(subset_10)

#os.rename('/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site.csv', '/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site10.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site10.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site10.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site10.csv')
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_n500_site10.png")
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site10.png")


In [ ]:
subset_11 = netCDF4.Dataset('n500_11.nc')
nearest_neighbor(subset_11, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site11_PLD30_settledist10.csv')
nearest_neighbor(subset_11, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site11_PLD60_settledist10.csv')
nearest_neighbor(subset_11, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site11_PLD90_settledist10.csv')
nearest_neighbor(subset_11, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site11_PLD180_settledist10.csv')
del(subset_11)

#os.rename('/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site.csv', '/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site11.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site11.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site11.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site11.csv')
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_n500_site11.png")
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site11.png")

In [ ]:
subset_12 = netCDF4.Dataset('n500_12.nc')
nearest_neighbor(subset_12, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site12_PLD30_settledist10.csv')
nearest_neighbor(subset_12, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site12_PLD60_settledist10.csv')
nearest_neighbor(subset_12, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site12_PLD90_settledist10.csv')
nearest_neighbor(subset_12, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site12_PLD180_settledist10.csv')
del(subset_12)

#os.rename('/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site.csv', '/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site12.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site12.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site12.csv')
#os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site12.csv')
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_n500_site12.png")
#os.rename("/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site.png","/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site12.png")

In [ ]:
subset7 = netCDF4.Dataset('n500_7.nc')
nearest_neighbor(subset7, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site7_PLD30_settledist10.csv')

In [ ]:
nearest_neighbor(subset7, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site7_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor(subset7, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site7_PLD90_settledist10.csv')

In [ ]:
def nearest_neighbor_divided(model_data, subsetting1,subsetting2, presettlement,PLD, settle_dist):
    # load the data
    lon = model_data.variables['lon'][subsetting1:subsetting2,:]
    lat = model_data.variables['lat'][subsetting1:subsetting2,:]
    age= model_data.variables['age'][subsetting1:subsetting2,:]
    site = model_data.variables['releaseSite'][subsetting1:subsetting2,:]
    trajectory = model_data.variables['trajectory'][subsetting1:subsetting2,:]
    
    # this just retrieves a label for each particle
    trajectory = trajectory[:,presettlement:PLD]
    labeled = np.reshape(trajectory, -1)
    trajectory = pd.DataFrame(trajectory)
    particle = trajectory.iloc[:, 0]
    del(trajectory)
    
    # remove the first 15 columns because those are before settling age and remove
    lon = lon[:,presettlement:PLD]
    lat = lat[:,presettlement:PLD]
    age = age[:,presettlement:PLD]
    
    site = pd.DataFrame(site)
    site_label = site.iloc[:, 0]
    del(site)
    
    #replace numbers with names of places
    site_label = site_label.replace(to_replace =1,value ="Guam")
    site_label = site_label.replace(to_replace =2,value ="Saipan")
    site_label = site_label.replace(to_replace =3,value ="Majuro")
    site_label = site_label.replace(to_replace =4,value ="Rongelap")
    site_label = site_label.replace(to_replace =5,value ="Kiritimati")
    site_label = site_label.replace(to_replace =6,value ="Pohnpei")
    site_label = site_label.replace(to_replace =7,value ="Okinawa")
    site_label = site_label.replace(to_replace =10,value ="FrenchFrigateShoal")
    site_label = site_label.replace(to_replace =11,value ="Johnston")
    site_label = site_label.replace(to_replace =12,value ="Wake")
    
    # convert to vectors
    lat = np.reshape(lat, -1)
    lon = np.reshape(lon, -1)
    age = np.reshape(age, -1)
    # convert to days from seconds
    age = age/86400
    
    # convert coordinates to radians
    rad_lon = np.deg2rad(lon)
    rad_lat = np.deg2rad(lat)
    del(lon)
    del(lat)
    
    # combine
    combined = np.vstack((rad_lon, rad_lat,labeled)).T
    del(rad_lat)
    del(rad_lon)
    del(labeled)
    coord = pd.DataFrame(data=combined, columns=['lat','lon','particle'])
    del(combined)
    coord['age'] = age
    del(age)
    
    # remove NAs so the next steps work
    coord = coord.dropna()
    
    # NOW, actually start search for nearest neighbor
    distances, indices = ball.query(coord[["lat", "lon"]].values, k = 1)
    
    # add data to dataframe
    coord['closest_point'] = indices
    coord['closest_distance'] = distances
    
    closest = coord.loc[coord.groupby('particle')['closest_distance'].idxmin()]
    del(coord)
    
    site_data = pd.DataFrame(data=[particle,site_label]).T
    del(site_label)
    del(particle)
    site_data.columns = ['particle','release_site']
    merge = closest.set_index('particle').join(site_data.set_index('particle'))
    
    #convert raidians to km
    merge['closest_dist_km'] = 6371 * merge['closest_distance']
    
    # add column of island code for settled
    points = merge['closest_point'].values
    island = []
    for x in points:
        add = land.loc[x,'island']
        island.append(add)
    
    # convert island code to name
    island_names = []
    for x in island:
        add = names.loc[x,'IslandName']
        island_names.append(add)
    merge['closest_island'] = island_names
    
    #make a column of settled or not if = or less than 5km and then graph the results and get summary statistics
    merge.loc[merge['closest_dist_km'] <= settle_dist, 'Settled'] = 'Settled' 
    merge.loc[merge['closest_dist_km'] > settle_dist, 'Settled'] = 'Not close enough'
    #merge.to_csv('/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site.csv', sep='\t')
    #settled_per_release_site = merge.groupby(['release_site', 'Settled']).size().reset_index(name='count')
    #settled_per_release_site.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site.csv', sep='\t')
    #settled_per_settled_site = merge.groupby(['closest_island', 'Settled']).size().reset_index(name='count')
    #settled_per_settled_site.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site.csv', sep='\t')
    
    # make dataset for each settled island to see where the ones settling are coming from
    settled_per_release_and_settledsite = merge.groupby(['release_site','closest_island', 'Settled']).size().reset_index(name='count')
    settled_per_release_and_settledsite.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv', sep='\t')
    #only_settled_per_release_and_settledsite = settled_per_release_and_settledsite[settled_per_release_and_settledsite['Settled'] == 'Settled'] 
    #only_settled_per_release_and_settledsite = only_settled_per_release_and_settledsite.drop(columns=['Settled'])
    
    #correct_format_to_plot_settlement = only_settled_per_release_and_settledsite.pivot_table(index="release_site", columns='closest_island', values='count')
    #correct_format_to_plot_settlement = correct_format_to_plot_settlement.fillna(0)
    
    #fig1, axs = plt.subplots(figsize=(20, 12))
    #ax = correct_format_to_plot_settlement.plot.bar(rot=0, ax=axs)
    #axs.set_ylabel("count")
    #axs.set_title('settlement to each island by release site')
    #fig1.savefig("/Users/taylorely/Documents/OceanModeling/settlement_n500_site.png")
    
    #fig2, axs = plt.subplots(figsize=(8, 50))
    #axes = correct_format_to_plot_settlement.plot.bar(rot=0, subplots=True, ax=axs)
    #axes[1].legend(loc=2)  
    #fig2.savefig("/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site.png")


In [ ]:
subset7 = netCDF4.Dataset('n500_7.nc')
nearest_neighbor_divided(subset7, 0, 1281000, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site7_part1_PLD180_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset7, 1281000, 2562000, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site7_part2_PLD180_settledist10.csv')

In [ ]:
site1 = pd.read_csv('settled_per_release_and_settledsite_n500_site1_PLD30_settledist10.csv', sep = '\t')
site2 = pd.read_csv('settled_per_release_and_settledsite_n500_site2_PLD30_settledist10.csv', sep = '\t')
site3 = pd.read_csv('settled_per_release_and_settledsite_n500_site3_PLD30_settledist10.csv', sep = '\t')
site4 = pd.read_csv('settled_per_release_and_settledsite_n500_site4_PLD30_settledist10.csv', sep = '\t')
site5 = pd.read_csv('settled_per_release_and_settledsite_n500_site5_PLD30_settledist10.csv', sep = '\t')
site6 = pd.read_csv('settled_per_release_and_settledsite_n500_site6_PLD30_settledist10.csv', sep = '\t')
site7 = pd.read_csv('settled_per_release_and_settledsite_n500_site7_PLD30_settledist10.csv', sep = '\t')
site10 = pd.read_csv('settled_per_release_and_settledsite_n500_site10_PLD30_settledist10.csv', sep = '\t')
site11 = pd.read_csv('settled_per_release_and_settledsite_n500_site11_PLD30_settledist10.csv', sep = '\t')
site12 = pd.read_csv('settled_per_release_and_settledsite_n500_site12_PLD30_settledist10.csv', sep = '\t')
merge_sites = pd.concat([site1,site2,site3,site4,site5,site6,site7,site10,site11,site12])
merge_sites.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_PLD30_settledist10.csv', sep='\t')

In [ ]:
site1 = pd.read_csv('settled_per_release_and_settledsite_n500_site1_PLD60_settledist10.csv', sep = '\t')
site2 = pd.read_csv('settled_per_release_and_settledsite_n500_site2_PLD60_settledist10.csv', sep = '\t')
site3 = pd.read_csv('settled_per_release_and_settledsite_n500_site3_PLD60_settledist10.csv', sep = '\t')
site4 = pd.read_csv('settled_per_release_and_settledsite_n500_site4_PLD60_settledist10.csv', sep = '\t')
site5 = pd.read_csv('settled_per_release_and_settledsite_n500_site5_PLD60_settledist10.csv', sep = '\t')
site6 = pd.read_csv('settled_per_release_and_settledsite_n500_site6_PLD60_settledist10.csv', sep = '\t')
site7 = pd.read_csv('settled_per_release_and_settledsite_n500_site7_PLD60_settledist10.csv', sep = '\t')
site10 = pd.read_csv('settled_per_release_and_settledsite_n500_site10_PLD60_settledist10.csv', sep = '\t')
site11 = pd.read_csv('settled_per_release_and_settledsite_n500_site11_PLD60_settledist10.csv', sep = '\t')
site12 = pd.read_csv('settled_per_release_and_settledsite_n500_site12_PLD60_settledist10.csv', sep = '\t')
merge_sites = pd.concat([site1,site2,site3,site4,site5,site6,site7,site10,site11,site12])
merge_sites.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_PLD60_settledist10.csv', sep='\t')

In [ ]:
site1 = pd.read_csv('settled_per_release_and_settledsite_n500_site1_PLD90_settledist10.csv', sep = '\t')
site2 = pd.read_csv('settled_per_release_and_settledsite_n500_site2_PLD90_settledist10.csv', sep = '\t')
site3 = pd.read_csv('settled_per_release_and_settledsite_n500_site3_PLD90_settledist10.csv', sep = '\t')
site4 = pd.read_csv('settled_per_release_and_settledsite_n500_site4_PLD90_settledist10.csv', sep = '\t')
site5 = pd.read_csv('settled_per_release_and_settledsite_n500_site5_PLD90_settledist10.csv', sep = '\t')
site6 = pd.read_csv('settled_per_release_and_settledsite_n500_site6_PLD90_settledist10.csv', sep = '\t')
site7 = pd.read_csv('settled_per_release_and_settledsite_n500_site7_PLD90_settledist10.csv', sep = '\t')
site10 = pd.read_csv('settled_per_release_and_settledsite_n500_site10_PLD90_settledist10.csv', sep = '\t')
site11 = pd.read_csv('settled_per_release_and_settledsite_n500_site11_PLD90_settledist10.csv', sep = '\t')
site12 = pd.read_csv('settled_per_release_and_settledsite_n500_site12_PLD90_settledist10.csv', sep = '\t')
merge_sites = pd.concat([site1,site2,site3,site4,site5,site6,site7,site10,site11,site12])
merge_sites.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_PLD90_settledist10.csv', sep='\t')

In [ ]:
site1 = pd.read_csv('settled_per_release_and_settledsite_n500_site1_PLD180_settledist10.csv', sep = '\t')
site2 = pd.read_csv('settled_per_release_and_settledsite_n500_site2_PLD180_settledist10.csv', sep = '\t')
site3 = pd.read_csv('settled_per_release_and_settledsite_n500_site3_PLD180_settledist10.csv', sep = '\t')
site4 = pd.read_csv('settled_per_release_and_settledsite_n500_site4_PLD180_settledist10.csv', sep = '\t')
site5 = pd.read_csv('settled_per_release_and_settledsite_n500_site5_PLD180_settledist10.csv', sep = '\t')
site6 = pd.read_csv('settled_per_release_and_settledsite_n500_site6_PLD180_settledist10.csv', sep = '\t')
site7 = pd.read_csv('settled_per_release_and_settledsite_n500_site7_part1_PLD180_settledist10.csv', sep = '\t')
site72 = pd.read_csv('settled_per_release_and_settledsite_n500_site7_part2_PLD180_settledist10.csv', sep = '\t')
site10 = pd.read_csv('settled_per_release_and_settledsite_n500_site10_PLD180_settledist10.csv', sep = '\t')
site11 = pd.read_csv('settled_per_release_and_settledsite_n500_site11_PLD180_settledist10.csv', sep = '\t')
site12 = pd.read_csv('settled_per_release_and_settledsite_n500_site12_PLD180_settledist10.csv', sep = '\t')
merge_sites = pd.concat([site1,site2,site3,site4,site5,site6,site7,site72,site10,site11,site12])
merge_sites.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_PLD180_settledist10.csv', sep='\t')

In [ ]:
subset_1000_1 = netCDF4.Dataset('n1000_1.nc')
nearest_neighbor(subset_1000_1, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site1_PLD30_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_1, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site1_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_1, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site1_PLD90_settledist10.csv')

In [ ]:
subset_1000_1 = netCDF4.Dataset('n1000_1.nc')
nearest_neighbor(subset_1000_1, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site1_PLD180_settledist10.csv')
del(subset_1000_1)

In [ ]:
subset_1000_2 = netCDF4.Dataset('n1000_2.nc')
nearest_neighbor(subset_1000_2, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site2_PLD30_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_2, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site2_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_2, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site2_PLD90_settledist10.csv')

In [ ]:
subset_1000_2 = netCDF4.Dataset('n1000_2.nc')
lon = subset_1000_2.variables['lon'][:]
print(lon.shape)

In [ ]:
nearest_neighbor_divided(subset_1000_2, 0,1098000, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site2_PLD180_part1_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset_1000_2,1098000,2196000, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site2_PLD180_part2_settledist10.csv')
del(subset_1000_2)

In [ ]:
subset_1000_3 = netCDF4.Dataset('n1000_3.nc')
nearest_neighbor(subset_1000_3, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site3_PLD30_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_3, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site3_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_3, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site3_PLD90_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_3, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site3_PLD180_settledist10.csv')
del(subset_1000_3)

In [ ]:
subset_1000_4 = netCDF4.Dataset('n1000_4.nc')
nearest_neighbor(subset_1000_4, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site4_PLD30_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_4, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site4_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_4, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site4_PLD90_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_4, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site4_PLD180_settledist10.csv')
del(subset_1000_4)

In [ ]:
subset_1000_5 = netCDF4.Dataset('n1000_5.nc')
nearest_neighbor(subset_1000_5, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site5_PLD30_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_5, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site5_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_5, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site5_PLD90_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_5, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site5_PLD180_settledist10.csv')
del(subset_1000_5)

In [ ]:
subset_1000_6 = netCDF4.Dataset('n1000_6.nc')
nearest_neighbor(subset_1000_6, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site6_PLD30_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_6, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site6_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_6, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site6_PLD90_settledist10.csv')

In [ ]:
subset_1000_6 = netCDF4.Dataset('n1000_6.nc')
lon6 = subset_1000_6.variables['lon'][:]
print(lon6.shape)

In [ ]:
del(lon6)
nearest_neighbor_divided(subset_1000_6,0,1098000, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site6_part1_PLD180_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset_1000_6,1098000,2196000, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site6_part2_PLD180_settledist10.csv')
del(subset_1000_6)

In [ ]:
subset_1000_7 = netCDF4.Dataset('LAPS_sensitivity_n1000_pld180_5m_Kh10_nday545_daily_07012012_first.nc')
lon2 = subset_1000_7.variables['lon'][:]
print(lon2.shape)
del(lon2)

In [ ]:
nearest_neighbor(subset_1000_7, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site7_PLD30_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset_1000_7,0,2562000, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site7_part1_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset_1000_7,2562000, 5124000,15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site7_part2_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset_1000_7,0,1708000,  15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site7_part1_PLD90_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset_1000_7,1708000,3416000, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site7_part2_PLD90_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset_1000_7, 3416000,5124000, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site7_part3_PLD90_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset_1000_7,0,1708000, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site7_part1_PLD180_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset_1000_7,1708000,3416000, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site7_part2_PLD180_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset_1000_7,3416000,5124000, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site7_part3_PLD180_settledist10.csv')
del(subset_1000_7)

In [ ]:
subset_1000_10 = netCDF4.Dataset('n1000_10.nc')
nearest_neighbor(subset_1000_10, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site10_PLD30_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_10, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site10_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_10, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site10_PLD90_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_10, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site10_PLD180_settledist10.csv')
del(subset_1000_10)

In [ ]:
subset_1000_11 = netCDF4.Dataset('n1000_11.nc')
nearest_neighbor(subset_1000_11, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site11_PLD30_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_11, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site11_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_11, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site11_PLD90_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_11, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site11_PLD180_settledist10.csv')
del(subset_1000_11)

In [ ]:
subset_1000_12 = netCDF4.Dataset('n1000_12.nc')
nearest_neighbor(subset_1000_12, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site12_PLD30_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_12, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site12_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor(subset_1000_12, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site12_PLD90_settledist10.csv')

In [ ]:
subset_1000_12 = netCDF4.Dataset('n1000_12.nc')
lon2 = subset_1000_12.variables['lon'][:]
print(lon2.shape)
del(lon2)

In [ ]:
nearest_neighbor_divided(subset_1000_12, 0,732000, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site12_part1_PLD180_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset_1000_12,732000,1464000, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_site12_part2_PLD180_settledist10.csv')
del(subset_1000_12)

In [ ]:
site1 = pd.read_csv('settled_per_release_and_settledsite_n1000_site1_PLD30_settledist10.csv', sep = '\t')
site2 = pd.read_csv('settled_per_release_and_settledsite_n1000_site2_PLD30_settledist10.csv', sep = '\t')
site3 = pd.read_csv('settled_per_release_and_settledsite_n1000_site3_PLD30_settledist10.csv', sep = '\t')
site4 = pd.read_csv('settled_per_release_and_settledsite_n1000_site4_PLD30_settledist10.csv', sep = '\t')
site5 = pd.read_csv('settled_per_release_and_settledsite_n1000_site5_PLD30_settledist10.csv', sep = '\t')
site6 = pd.read_csv('settled_per_release_and_settledsite_n1000_site6_PLD30_settledist10.csv', sep = '\t')
site7 = pd.read_csv('settled_per_release_and_settledsite_n1000_site7_PLD30_settledist10.csv', sep = '\t')
site10 = pd.read_csv('settled_per_release_and_settledsite_n1000_site10_PLD30_settledist10.csv', sep = '\t')
site11 = pd.read_csv('settled_per_release_and_settledsite_n1000_site11_PLD30_settledist10.csv', sep = '\t')
site12 = pd.read_csv('settled_per_release_and_settledsite_n1000_site12_PLD30_settledist10.csv', sep = '\t')
merge_sites = pd.concat([site1,site2,site3,site4,site5,site6,site7,site10,site11,site12])
merge_sites.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_PLD30_settledist10.csv', sep='\t')
del(merge_sites)

In [ ]:
site1 = pd.read_csv('settled_per_release_and_settledsite_n1000_site1_PLD60_settledist10.csv', sep = '\t')
site2 = pd.read_csv('settled_per_release_and_settledsite_n1000_site2_PLD60_settledist10.csv', sep = '\t')
site3 = pd.read_csv('settled_per_release_and_settledsite_n1000_site3_PLD60_settledist10.csv', sep = '\t')
site4 = pd.read_csv('settled_per_release_and_settledsite_n1000_site4_PLD60_settledist10.csv', sep = '\t')
site5 = pd.read_csv('settled_per_release_and_settledsite_n1000_site5_PLD60_settledist10.csv', sep = '\t')
site6 = pd.read_csv('settled_per_release_and_settledsite_n1000_site6_PLD60_settledist10.csv', sep = '\t')
site7 = pd.read_csv('settled_per_release_and_settledsite_n1000_site7_part1_PLD60_settledist10.csv', sep = '\t')
site72 = pd.read_csv('settled_per_release_and_settledsite_n1000_site7_part2_PLD60_settledist10.csv', sep = '\t')
site10 = pd.read_csv('settled_per_release_and_settledsite_n1000_site10_PLD60_settledist10.csv', sep = '\t')
site11 = pd.read_csv('settled_per_release_and_settledsite_n1000_site11_PLD60_settledist10.csv', sep = '\t')
site12 = pd.read_csv('settled_per_release_and_settledsite_n1000_site12_PLD60_settledist10.csv', sep = '\t')
merge_sites = pd.concat([site1,site2,site3,site4,site5,site6,site7,site72,site10,site11,site12])
merge_sites.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_PLD60_settledist10.csv', sep='\t')
del(merge_sites)

In [ ]:
site1 = pd.read_csv('settled_per_release_and_settledsite_n1000_site1_PLD90_settledist10.csv', sep = '\t')
site2 = pd.read_csv('settled_per_release_and_settledsite_n1000_site2_PLD90_settledist10.csv', sep = '\t')
site3 = pd.read_csv('settled_per_release_and_settledsite_n1000_site3_PLD90_settledist10.csv', sep = '\t')
site4 = pd.read_csv('settled_per_release_and_settledsite_n1000_site4_PLD90_settledist10.csv', sep = '\t')
site5 = pd.read_csv('settled_per_release_and_settledsite_n1000_site5_PLD90_settledist10.csv', sep = '\t')
site6 = pd.read_csv('settled_per_release_and_settledsite_n1000_site6_PLD90_settledist10.csv', sep = '\t')
site7 = pd.read_csv('settled_per_release_and_settledsite_n1000_site7_part1_PLD90_settledist10.csv', sep = '\t')
site72 = pd.read_csv('settled_per_release_and_settledsite_n1000_site7_part2_PLD90_settledist10.csv', sep = '\t')
site73 = pd.read_csv('settled_per_release_and_settledsite_n1000_site7_part3_PLD90_settledist10.csv', sep = '\t')
site10 = pd.read_csv('settled_per_release_and_settledsite_n1000_site10_PLD90_settledist10.csv', sep = '\t')
site11 = pd.read_csv('settled_per_release_and_settledsite_n1000_site11_PLD90_settledist10.csv', sep = '\t')
site12 = pd.read_csv('settled_per_release_and_settledsite_n1000_site12_PLD90_settledist10.csv', sep = '\t')
merge_sites = pd.concat([site1,site2,site3,site4,site5,site6,site7,site72,site73,site10,site11,site12])
merge_sites.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_PLD90_settledist10.csv', sep='\t')
del(merge_sites)

In [ ]:
site1 = pd.read_csv('settled_per_release_and_settledsite_n1000_site1_PLD180_settledist10.csv', sep = '\t')
site2 = pd.read_csv('settled_per_release_and_settledsite_n1000_site2_PLD180_part1_settledist10.csv', sep = '\t')
site22 = pd.read_csv('settled_per_release_and_settledsite_n1000_site2_PLD180_part2_settledist10.csv', sep = '\t')
site3 = pd.read_csv('settled_per_release_and_settledsite_n1000_site3_PLD180_settledist10.csv', sep = '\t')
site4 = pd.read_csv('settled_per_release_and_settledsite_n1000_site4_PLD180_settledist10.csv', sep = '\t')
site5 = pd.read_csv('settled_per_release_and_settledsite_n1000_site5_PLD180_settledist10.csv', sep = '\t')
site6 = pd.read_csv('settled_per_release_and_settledsite_n1000_site6_part1_PLD180_settledist10.csv', sep = '\t')
site62 = pd.read_csv('settled_per_release_and_settledsite_n1000_site6_part2_PLD180_settledist10.csv', sep = '\t')
site7 = pd.read_csv('settled_per_release_and_settledsite_n1000_site7_part1_PLD180_settledist10.csv', sep = '\t')
site72 = pd.read_csv('settled_per_release_and_settledsite_n1000_site7_part2_PLD180_settledist10.csv', sep = '\t')
site73 = pd.read_csv('settled_per_release_and_settledsite_n1000_site7_part3_PLD180_settledist10.csv', sep = '\t')
site10 = pd.read_csv('settled_per_release_and_settledsite_n1000_site10_PLD180_settledist10.csv', sep = '\t')
site11 = pd.read_csv('settled_per_release_and_settledsite_n1000_site11_PLD180_settledist10.csv', sep = '\t')
site12 = pd.read_csv('settled_per_release_and_settledsite_n1000_site12_part1_PLD180_settledist10.csv', sep = '\t')
site122 = pd.read_csv('settled_per_release_and_settledsite_n1000_site12_part2_PLD180_settledist10.csv', sep = '\t')
merge_sites = pd.concat([site1,site2,site22,site3,site4,site5,site6,site62,site7,site72,site73,site10,site11,site12,site122])
merge_sites.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n1000_PLD180_settledist10.csv', sep='\t')
del(merge_sites)

In [ ]:
subset_250 = netCDF4.Dataset('LAPS_sensitivity_n250_pld180_5m_Kh10_nday545_daily_07012012.nc')
nearest_neighbor(subset_250, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n250_PLD30_settledist10.csv')

In [ ]:
nearest_neighbor(subset_250, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n250_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor(subset_250, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n250_PLD90_settledist10.csv')

In [ ]:
subset_250  = netCDF4.Dataset('LAPS_sensitivity_n250_pld180_5m_Kh10_nday545_daily_07012012.nc')
lon2 = subset_250.variables['lon'][:]
print(lon2.shape)
del(lon2)

In [ ]:
nearest_neighbor_divided(subset_250, 0, 1875750, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n250_PLD180_part1_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset_250,1875750,3751500, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n250_PLD180_part2_settledist10.csv')

In [ ]:
del(subset_250)

In [ ]:
site1 = pd.read_csv('settled_per_release_and_settledsite_n250_PLD180_part1_settledist10.csv', sep = '\t')
site2 = pd.read_csv('settled_per_release_and_settledsite_n250_PLD180_part2_settledist10.csv', sep = '\t')
merge_sites = pd.concat([site1,site2])
merge_sites.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n250_PLD180_settledist10.csv', sep='\t')
del(merge_sites)

In [ ]:
subset_750_first = netCDF4.Dataset('LAPS_sensitivity_n750_pld180_5m_Kh10_nday545_daily_07012012_first.nc')
nearest_neighbor(subset_750_first, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_first_PLD30_settledist10.csv')

In [ ]:
nearest_neighbor(subset_750_first, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_first_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor(subset_750_first, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_first_PLD90_settledist10.csv')

In [ ]:
lon2 = subset_750_first.variables['lon'][:]
print(lon2.shape)
del(lon2)

In [ ]:
nearest_neighbor_divided(subset_750_first,0,1921500, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_first_PLD180_part1_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset_750_first,1921500,3843000, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_first_PLD180_part2_settledist10.csv')

In [ ]:
del(subset_750_first)

In [ ]:
subset_750_second = netCDF4.Dataset('LAPS_sensitivity_n750_pld180_5m_Kh10_nday545_daily_07012012_second.nc')
nearest_neighbor(subset_750_second, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_second_PLD30_settledist10.csv')

In [ ]:
nearest_neighbor(subset_750_second, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_second_PLD60_settledist10.csv')

In [ ]:
nearest_neighbor(subset_750_second, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_second_PLD90_settledist10.csv')

In [ ]:
lon2 = subset_750_second.variables['lon'][:]
print(lon2.shape)
del(lon2)

In [ ]:
nearest_neighbor_divided(subset_750_second,0,1921500, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_second_PLD180_part1_settledist10.csv')

In [ ]:
subset_750_second = netCDF4.Dataset('LAPS_sensitivity_n750_pld180_5m_Kh10_nday545_daily_07012012_second.nc')
nearest_neighbor_divided(subset_750_second,1921500,3843000, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_second_PLD180_part2_settledist10.csv')

In [ ]:
del(subset_750_second)

In [ ]:
subset_750_third = netCDF4.Dataset('LAPS_sensitivity_n750_pld180_5m_Kh10_nday545_daily_07012012_third.nc')
nearest_neighbor(subset_750_third, 15, 30, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_third_PLD30_settledist10.csv')

In [ ]:
nearest_neighbor(subset_750_third, 15, 60, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_third_PLD60_settledist10.csv')

In [ ]:
subset_750_third = netCDF4.Dataset('LAPS_sensitivity_n750_pld180_5m_Kh10_nday545_daily_07012012_third.nc')
nearest_neighbor_divided(subset_750_third,0,1784250, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_third_PLD90_part1_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset_750_third,1784250,3568500, 15, 90, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_third_PLD90_part2_settledist10.csv')

In [ ]:
lon2 = subset_750_third.variables['lon'][:]
print(lon2.shape)
del(lon2)

In [ ]:
nearest_neighbor_divided(subset_750_third,0,1784250, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_third_PLD180_part1_settledist10.csv')

In [ ]:
nearest_neighbor_divided(subset_750_third,1784250,3568500, 15, 180, 10)
os.rename('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site.csv','/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_third_PLD180_part2_settledist10.csv')

In [ ]:
del(subset_750_third)

In [ ]:
site1 = pd.read_csv('settled_per_release_and_settledsite_n750_first_PLD30_settledist10.csv', sep = '\t')
site2 = pd.read_csv('settled_per_release_and_settledsite_n750_second_PLD30_settledist10.csv', sep = '\t')
site3 = pd.read_csv('settled_per_release_and_settledsite_n750_third_PLD30_settledist10.csv', sep = '\t')
merge_sites = pd.concat([site1,site2,site3])
merge_sites.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_PLD30_settledist10.csv', sep='\t')
del(merge_sites)

In [ ]:
site1 = pd.read_csv('settled_per_release_and_settledsite_n750_first_PLD60_settledist10.csv', sep = '\t')
site2 = pd.read_csv('settled_per_release_and_settledsite_n750_second_PLD60_settledist10.csv', sep = '\t')
site3 = pd.read_csv('settled_per_release_and_settledsite_n750_third_PLD60_settledist10.csv', sep = '\t')
merge_sites = pd.concat([site1,site2,site3])
merge_sites.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_PLD60_settledist10.csv', sep='\t')
del(merge_sites)

In [ ]:
site1 = pd.read_csv('settled_per_release_and_settledsite_n750_first_PLD90_settledist10.csv', sep = '\t')
site2 = pd.read_csv('settled_per_release_and_settledsite_n750_second_PLD90_settledist10.csv', sep = '\t')
site3 = pd.read_csv('settled_per_release_and_settledsite_n750_third_PLD90_part1_settledist10.csv', sep = '\t')
site4 = pd.read_csv('settled_per_release_and_settledsite_n750_third_PLD90_part2_settledist10.csv', sep = '\t')
merge_sites = pd.concat([site1,site2,site3,site4])
merge_sites.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_PLD90_settledist10.csv', sep='\t')
del(merge_sites)

In [ ]:
site1 = pd.read_csv('settled_per_release_and_settledsite_n750_first_PLD180_part1_settledist10.csv', sep = '\t')
site2 = pd.read_csv('settled_per_release_and_settledsite_n750_first_PLD180_part2_settledist10.csv', sep = '\t')
site22 = pd.read_csv('settled_per_release_and_settledsite_n750_second_PLD180_part1_settledist10.csv', sep = '\t')
site3 = pd.read_csv('settled_per_release_and_settledsite_n750_second_PLD180_part2_settledist10.csv', sep = '\t')
site4 = pd.read_csv('settled_per_release_and_settledsite_n750_third_PLD180_part1_settledist10.csv', sep = '\t')
site5 = pd.read_csv('settled_per_release_and_settledsite_n750_third_PLD180_part2_settledist10.csv', sep = '\t')
merge_sites = pd.concat([site1,site2,site22,site3,site4,site5])
merge_sites.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n750_PLD180_settledist10.csv', sep='\t')
del(merge_sites)

# The rest of the cells are just leftover code that might be useful but no longer run for this analysis

In [ ]:
data_netcdf4_n500 = netCDF4.Dataset('n500_7.nc')

In [ ]:
# load site 1 data
lon_netcdf4_n500_1 = data_netcdf4_n500.variables['lon'][:]
lat_netcdf4_n500_1 = data_netcdf4_n500.variables['lat'][:]
age_netcdf4_n500_1 = data_netcdf4_n500.variables['age'][:]
site_no_netcdf4_n500_1 = data_netcdf4_n500.variables['releaseSite'][:]
trajectory_netcdf4_n500_1 = data_netcdf4_n500.variables['trajectory'][:]

In [ ]:
lon_1 = lon_netcdf4_n500_1[:1281000,:]
lon_2 = lon_netcdf4_n500_1[1281000:,:]
lat_1 = lat_netcdf4_n500_1[:1281000,:]
lat_2 = lat_netcdf4_n500_1[1281000:,:]
age_1 = age_netcdf4_n500_1[:1281000,:]
age_2 = age_netcdf4_n500_1[1281000:,:]
site_1 = site_no_netcdf4_n500_1[:1281000,:]
site_2 = site_no_netcdf4_n500_1[1281000:,:]
traj_1 = trajectory_netcdf4_n500_1[:1281000,:]
traj_2 = trajectory_netcdf4_n500_1[1281000:,:]

In [ ]:
del(data_netcdf4_n500)

In [ ]:
# this just retrieves a label for each particle
traj_1 = traj_1[:,15:]
labeled_1 = np.reshape(traj_1, -1)
traj_1 = pd.DataFrame(traj_1)
particle_1 = traj_1.iloc[:, 0]
del(traj_1)
traj_2 = traj_2[:,15:]
labeled_2 = np.reshape(traj_2, -1)
traj_2 = pd.DataFrame(traj_2)
particle_2 = traj_2.iloc[:, 0]
del(traj_2)
# make sure to delete things that won't be used again so it reduces load on memory

In [ ]:
# remove the first 20 columns because those are before settling age
lon_1 = lon_1[:,15:]
lon_2 = lon_2[:,15:]

In [ ]:
lat_1 = lat_1[:,15:]
lat_2 = lat_2[:,15:]

In [ ]:
age_1 = age_1[:,15:]
age_2 = age_2[:,15:]

In [ ]:
site_1 = pd.DataFrame(site_1)
site_label_1 = site_1.iloc[:, 0]
del(site_1)
site_2 = pd.DataFrame(site_2)
site_label_2 = site_2.iloc[:, 0]
del(site_2)
site_label_1 = site_label_1.replace(to_replace =7,value ="Okinawa")
site_label_2 = site_label_2.replace(to_replace =7,value ="Okinawa")

In [ ]:
lat_1 = np.reshape(lat_1, -1)
lon_1 = np.reshape(lon_1, -1)
age_1 = np.reshape(age_1, -1)
age_1 = age_1/86400

In [ ]:
# convert coordinates to radians
rad_lon_1 = np.deg2rad(lon_1)
rad_lat_1 = np.deg2rad(lat_1)
del(lon_1)
del(lat_1)

In [ ]:
combined_1 = np.vstack((rad_lon_1, rad_lat_1,labeled_1)).T
del(rad_lat_1)
del(rad_lon_1)
del(labeled_1)

In [ ]:
coord_1 = pd.DataFrame(data=combined_1, columns=['lat','lon','particle'])
del(combined_1)
coord_1['age'] = age_1
del(age_1)
coord_1 = coord_1.dropna()

In [ ]:
distances, indices = ball.query(coord_1[["lat", "lon"]].values, k = 1)

In [ ]:
# add data to dataframe
coord_1['closest_point'] = indices
coord_1['closest_distance'] = distances

In [ ]:
closest_1 = coord_1.loc[coord_1.groupby('particle')['closest_distance'].idxmin()]
del(coord_1)
del(indices)
del(distances)

In [ ]:
site_data_1 = pd.DataFrame(data=[particle_1,site_label_1]).T
del(site_label_1)
del(particle_1)
site_data_1.columns = ['particle','release_site']
merge_1 = closest_1.set_index('particle').join(site_data_1.set_index('particle'))
#convert raidians to km
merge_1['closest_dist_km'] = 6371 * merge_1['closest_distance']

In [ ]:
# add column of island code for settled
points = merge_1['closest_point'].values
island = []
for x in points:
    add = land.loc[x,'island']
    island.append(add)
    
# convert island code to name
island_names = []
for x in island:
    add = names.loc[x,'IslandName']
    island_names.append(add)
merge_1['closest_island'] = island_names

In [ ]:
lat_2 = np.reshape(lat_2, -1)
lon_2 = np.reshape(lon_2, -1)
age_2 = np.reshape(age_2, -1)
age_2 = age_2/86400

In [ ]:
# convert coordinates to radians
rad_lon_2 = np.deg2rad(lon_2)
rad_lat_2 = np.deg2rad(lat_2)
del(lon_2)
del(lat_2)

In [ ]:
combined_2 = np.vstack((rad_lon_2, rad_lat_2,labeled_2)).T
del(rad_lat_2)
del(rad_lon_2)
del(labeled_2)

In [ ]:
coord_2 = pd.DataFrame(data=combined_2, columns=['lat','lon','particle'])
del(combined_2)
coord_2['age'] = age_2
del(age_2)
coord_2 = coord_2.dropna()

In [ ]:
distances, indices = ball.query(coord_2[["lat", "lon"]].values, k = 1)

In [ ]:
# add data to dataframe
coord_2['closest_point'] = indices
coord_2['closest_distance'] = distances

In [ ]:
closest_2 = coord_2.loc[coord_2.groupby('particle')['closest_distance'].idxmin()]
del(coord_2)
del(indices)
del(distances)

In [ ]:
site_data_2 = pd.DataFrame(data=[particle_2,site_label_2]).T
del(site_label_2)
del(particle_2)
site_data_2.columns = ['particle','release_site']
merge_2 = closest_2.set_index('particle').join(site_data_2.set_index('particle'))
#convert raidians to km
merge_2['closest_dist_km'] = 6371 * merge_2['closest_distance']

In [ ]:
# add column of island code for settled
points = merge_2['closest_point'].values
island = []
for x in points:
    add = land.loc[x,'island']
    island.append(add)
    
# convert island code to name
island_names = []
for x in island:
    add = names.loc[x,'IslandName']
    island_names.append(add)
merge_2['closest_island'] = island_names

In [ ]:
merge = pd.concat([merge_1,merge_2])

In [ ]:
#make a column of settled or not if = or less than 5km and then graph the results and get summary statistics
merge.loc[merge['closest_dist_km'] <= 10, 'Settled'] = 'Settled' 
merge.loc[merge['closest_dist_km'] > 10, 'Settled'] = 'Not close enough'
merge.to_csv('/Users/taylorely/Documents/OceanModeling/nearest_island_and_settled_n500_site7.csv', sep='\t')
# settled_per_release_site = merge.groupby(['release_site', 'Settled']).size().reset_index(name='count')
# settled_per_release_site.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per__release_site_n500_site7.csv', sep='\t')
# settled_per_settled_site = merge.groupby(['closest_island', 'Settled']).size().reset_index(name='count')
# settled_per_settled_site.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_settled_site_n500_site7.csv', sep='\t')
    
# make dataset for each settled island to see where the ones settling are coming from
settled_per_release_and_settledsite = merge.groupby(['release_site','closest_island', 'Settled']).size().reset_index(name='count')
settled_per_release_and_settledsite.to_csv('/Users/taylorely/Documents/OceanModeling/settled_per_release_and_settledsite_n500_site7_PLD30_settledist10.csv', sep='\t')
#only_settled_per_release_and_settledsite = settled_per_release_and_settledsite[settled_per_release_and_settledsite['Settled'] == 'Settled'] 
#only_settled_per_release_and_settledsite = only_settled_per_release_and_settledsite.drop(columns=['Settled'])
    
#correct_format_to_plot_settlement = only_settled_per_release_and_settledsite.pivot_table(index="release_site", 
                    columns='closest_island', 
                    values='count')
#correct_format_to_plot_settlement = correct_format_to_plot_settlement.fillna(0)

#fig1, axs = plt.subplots(figsize=(20, 12))
#ax = correct_format_to_plot_settlement.plot.bar(rot=0, ax=axs)
#axs.set_ylabel("count")
#axs.set_title('settlement to each island by release site')
#fig1.savefig("/Users/taylorely/Documents/OceanModeling/settlement_n500_site7.png")
    
#fig2, axs = plt.subplots(figsize=(8, 50))
#axes = correct_format_to_plot_settlement.plot.bar(rot=0, subplots=True, ax=axs)
#axes[1].legend(loc=2)  
#fig2.savefig("/Users/taylorely/Documents/OceanModeling/settlement_subplot_n500_site7.png")

#Make graphs with everything